<a href="https://colab.research.google.com/github/JuTsgo01/Analise_Preco_Combustivel/blob/main/analise_combustivel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalando aa bibliotecas e importando
!pip install pandasql
!pip install sqlalchemy
!pip install matplotlib

import pandas as pd
from pandasql import sqldf
import matplotlib.pyplot as plt

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=5af22be4cf6103926fa3f3591f7175dd8c11b5327892b463acb45e9fc8e8ade9
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql




---



In [2]:
df = pd.read_csv('/content/drive/MyDrive/ESTUDOS/analises/combustivel/precos_semestrais.csv', sep=';')
df.head()

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,02.817.655/0001-82,RODOVIA AL-220,1901,NaN,SANTA ESMERALDA,57312-025,GASOLINA,01/01/2024,"5,68",NaN,R$ / litro,VIBRA
1,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,02.817.655/0001-82,RODOVIA AL-220,1901,NaN,SANTA ESMERALDA,57312-025,GASOLINA ADITIVADA,01/01/2024,"5,68",NaN,R$ / litro,VIBRA
2,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,02.817.655/0001-82,RODOVIA AL-220,1901,NaN,SANTA ESMERALDA,57312-025,DIESEL S10,01/01/2024,"5,93",NaN,R$ / litro,VIBRA
3,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,02.817.655/0001-82,RODOVIA AL-220,1901,NaN,SANTA ESMERALDA,57312-025,ETANOL,01/01/2024,"3,99",NaN,R$ / litro,VIBRA
4,NE,AL,ARAPIRACA,AUTO POSTO SAO FRANCISCO LTDA,12.706.529/0001-04,RUA SAO FRANCISCO,274,NaN,CENTRO,57300-080,GASOLINA,01/01/2024,"5,48",NaN,R$ / litro,VIBRA


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477154 entries, 0 to 477153
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Regiao - Sigla     477154 non-null  object 
 1   Estado - Sigla     477154 non-null  object 
 2   Municipio          477154 non-null  object 
 3   Revenda            477154 non-null  object 
 4   CNPJ da Revenda    477154 non-null  object 
 5   Nome da Rua        477154 non-null  object 
 6   Numero Rua         477089 non-null  object 
 7   Complemento        110830 non-null  object 
 8   Bairro             476286 non-null  object 
 9   Cep                477154 non-null  object 
 10  Produto            477154 non-null  object 
 11  Data da Coleta     477154 non-null  object 
 12  Valor de Venda     477154 non-null  object 
 13  Valor de Compra    0 non-null       float64
 14  Unidade de Medida  477154 non-null  object 
 15  Bandeira           477154 non-null  object 
dtypes:

###Nesse caso, não precisaremos tratar as colunas não nulas, pois usaremos as colunas onde não tem valores nulos, por exemplo, região, estado, municipio, revenda, produto, data da coleta, valor da venda e bandeira.



---



In [4]:
columns_to_drop = ['CNPJ da Revenda', 'Nome da Rua', 'Numero Rua',
                    'Complemento', 'Bairro', 'Cep', 'Valor de Compra', 'Unidade de Medida']

df = df.drop(columns=columns_to_drop)

In [5]:
#Tratando os nomes das colunas
df.columns = df.columns.str.strip().str.lower().str.replace('_', '').str.replace(r'[^a-zA-Z0-9_]', '', regex=True)

In [6]:
#Fazendo um tratamento do dado para data no formato correto
df['datadacoleta'] = pd.to_datetime(df['datadacoleta'], format='%d/%m/%Y')

#Ajustando a coluna que era do tipo objeto para float e trocando vírgula por pont
df['valordevenda'] = df['valordevenda'].str.replace(',', '.').astype(float)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477154 entries, 0 to 477153
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   regiaosigla   477154 non-null  object        
 1   estadosigla   477154 non-null  object        
 2   municipio     477154 non-null  object        
 3   revenda       477154 non-null  object        
 4   produto       477154 non-null  object        
 5   datadacoleta  477154 non-null  datetime64[ns]
 6   valordevenda  477154 non-null  float64       
 7   bandeira      477154 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 29.1+ MB


Com a execução acima podemos ver que não há nenhum dado nulo no dataframe

In [8]:
df.head()

,regiaosigla,estadosigla,municipio,revenda,produto,datadacoleta,valordevenda,bandeira
0,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,GASOLINA,2024-01-01,5.68,VIBRA
1,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,GASOLINA ADITIVADA,2024-01-01,5.68,VIBRA
2,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,DIESEL S10,2024-01-01,5.93,VIBRA
3,NE,AL,ARAPIRACA,COMERCIAL DE COMBUSTIVEIS E LUBRIFICANTES VITA...,ETANOL,2024-01-01,3.99,VIBRA
4,NE,AL,ARAPIRACA,AUTO POSTO SAO FRANCISCO LTDA,GASOLINA,2024-01-01,5.48,VIBRA


Feito o tratamento dos dados que usaremos, limpeza das colunas que não usaremos na análise e também nos nomes das colunas, seguiremos para a análise

In [9]:
df.to_excel('df_tratado.xlsx', index=False)



---


#***Análise do produto gasolina***






---



### Query 1 - Preço Máximo, Mínimo e Médio Nacional

In [10]:
gasolina_max_min_avg = """

SELECT
  t1.produto AS produto
  , ROUND(AVG(t1.valordevenda), 2) AS preco_medio
  , MIN(t1.valordevenda) AS preco_minimo
  , MAX(t1.valordevenda) AS preco_maximo

FROM
  df AS t1

WHERE 1=1
  AND t1.produto = 'GASOLINA'

ORDER BY
  AVG(t1.valordevenda) DESC

"""
sqldf(gasolina_max_min_avg)

,produto,preco_medio,preco_minimo,preco_maximo
0,GASOLINA,5.78,4.55,7.99




---



### Query 2 - Análise apontando o preço médio nacional por mês

In [11]:
gasolina_mes_max_min_avg = """

SELECT
  strftime('%m', t1.datadacoleta) AS mes
  , ROUND(AVG(t1.valordevenda), 2) AS preco_medio
  , MIN(t1.valordevenda) AS preco_minimo
  , MAX(t1.valordevenda) AS preco_maximo

FROM
  df AS t1

WHERE 1=1
  AND t1.produto = 'GASOLINA'

GROUP BY
  strftime('%m', t1.datadacoleta)

ORDER BY
  AVG(t1.valordevenda) DESC

"""
sqldf(gasolina_mes_max_min_avg)

,mes,preco_medio,preco_minimo,preco_maximo
0,05,5.87,4.59,7.97
1,06,5.86,4.59,7.97
2,04,5.82,4.69,7.97
3,02,5.77,4.59,7.97
4,03,5.77,4.59,7.97
5,01,5.61,4.55,7.99




---



### Query 3 - Análise apontando em qual mês apresenta os menores preços médios da gasolina por estado brasileiro

In [12]:
query = """

WITH tb_avg_estado_mes AS (

    SELECT
      t1.estadosigla AS estado
      , strftime('%m', t1.datadacoleta) AS mes
      , ROUND(AVG(t1.valordevenda), 2) AS preco_medio


    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla
      , strftime('%m', t1.datadacoleta)


    ORDER BY
      AVG(t1.valordevenda) DESC
)

SELECT
  t2.estado
  , t2.mes
  , MIN(t2.preco_medio) menor_preco_medio

FROM
  tb_avg_estado_mes AS t2

GROUP BY
  t2.estado

"""
sqldf(query)

,estado,mes,menor_preco_medio
0,AC,01,6.88
1,AL,01,5.72
2,AM,01,6.08
3,AP,01,5.48
4,BA,01,5.83
5,CE,04,5.88
6,DF,01,5.46
7,ES,01,5.60
8,GO,01,5.62
9,MA,01,5.46




---



### Query 4 - Análise apontando em qual mês apresenta os maiores preços médios da gasolina por estado brasileiro

In [13]:
query = """

WITH tb_avg_estado_mes AS (

    SELECT
      t1.estadosigla AS estado
      , strftime('%m', t1.datadacoleta) AS mes
      , ROUND(AVG(t1.valordevenda), 2) AS preco_medio


    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla
      , strftime('%m', t1.datadacoleta)


    ORDER BY
      AVG(t1.valordevenda) DESC
)

SELECT
  t2.estado
  , t2.mes
  , MAX(t2.preco_medio) maior_preco_medio

FROM
  tb_avg_estado_mes AS t2

GROUP BY
  t2.estado

"""
sqldf(query)

,estado,mes,maior_preco_medio
0,AC,06,7.04
1,AL,06,6.08
2,AM,03,6.59
3,AP,06,5.65
4,BA,05,6.24
5,CE,05,6.02
6,DF,06,5.87
7,ES,05,6.02
8,GO,04,5.85
9,MA,06,5.84




---



### Query 5 - Análise apontando o preço médio, máximo e mínimo por estado e mostrando a diferença percentual e absoluta entre o preço máximo e míninmo

In [14]:
query = """

WITH tb_valores_geral AS (

    SELECT
      t1.estadosigla AS estado
      , ROUND(AVG(t1.valordevenda),2) AS preco_medio
      , MIN(t1.valordevenda) AS preco_minimo
      , MAX(t1.valordevenda) AS preco_maximo

    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla

    ORDER BY
      AVG(t1.valordevenda) DESC
)

SELECT
  t2.*
  ,ROUND((1-(t2.preco_minimo/t2.preco_maximo))*100,2) AS diff_percent_max_min
  , t2.preco_maximo - preco_minimo AS diff_absoluta_max_min

FROM
  tb_valores_geral AS t2

"""
sqldf(query)

,estado,preco_medio,preco_minimo,preco_maximo,diff_percent_max_min,diff_absoluta_max_min
0,AC,6.99,6.49,7.63,14.94,1.14
1,AM,6.44,5.19,7.80,33.46,2.61
2,RO,6.39,5.12,7.09,27.79,1.97
3,RR,6.10,5.87,6.42,8.57,0.55
4,RN,6.07,5.39,6.49,16.95,1.10
5,BA,6.06,5.09,6.99,27.18,1.90
6,SE,6.00,4.98,6.49,23.27,1.51
7,PA,5.96,4.74,6.99,32.19,2.25
8,CE,5.95,5.14,6.99,26.47,1.85
9,AL,5.91,5.28,6.69,21.08,1.41




---



### Query 6 - Análise apontando a diferença percentual e absoluta de forma descrescente (entre o mais caro e o segundo mais caro, entre o segundo mais caro e o terceiro mais caro, entre o n mais caro e o n-1 mais caro)

In [15]:
query = """

WITH tb_lead AS(

    SELECT
      t1.estadosigla AS estado
      , ROUND(AVG(t1.valordevenda),2) AS preco_medio
      , LEAD(ROUND(AVG(t1.valordevenda),2), 1) OVER (ORDER BY ROUND(AVG(t1.valordevenda),2) DESC) AS lead_preco_medio

    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla
)

SELECT
  t2.estado,
  ROUND((1-(t2.lead_preco_medio / t2.preco_medio))*100, 2) AS diff_percent
  , t2.preco_medio - t2.lead_preco_medio AS diff_absoluta

FROM
  tb_lead AS t2

"""
sqldf(query)

,estado,diff_percent,diff_absoluta
0,AC,7.87,0.55
1,AM,0.78,0.05
2,RO,4.54,0.29
3,RR,0.49,0.03
4,RN,0.16,0.01
5,BA,0.99,0.06
6,SE,0.67,0.04
7,PA,0.17,0.01
8,CE,0.67,0.04
9,AL,0.34,0.02




---



### Query 7 - Análise apontando a diferença absoluta e percentual dos preços médio que a gasolina já foi vendida no estado com maior preço médio (Acre - AC) e o de menor preço médio (Piauí - PI)

In [29]:
query = """

WITH tb_regiao_metricas AS (

    SELECT
      t1.estadosigla AS estado
      , ROUND(AVG(t1.valordevenda),2) AS preco_medio
      , MIN(t1.valordevenda) AS preco_minimo
      , MAX(t1.valordevenda) AS preco_maximo

    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla

    ORDER BY
      AVG(t1.valordevenda) DESC
)
SELECT
  ROUND((1-(t3.preco_medio/t2.preco_medio))*100, 2) AS diff_percent
  , (t2.preco_medio-t3.preco_medio) AS diff_absoluta

FROM
  tb_regiao_metricas AS t2

JOIN
  tb_regiao_metricas AS t3

ON
  t2.estado = 'AC' AND t3.estado = 'PI'

"""
sqldf(query)

,diff_percent,diff_absoluta
0,20.31,1.42


Há uma diferença de 20% e R$ 1,42 entre os preços médio que a gasolina já foi vendida no estado com maior preço médio e o estado com menor preço médio



---



### Query 8 - Análise apontando a diferença absoluta e percentual dos preços máximo que a gasolina já foi vendida no estado com maior preço médio (Acre - AC) e o de menor preço médio (Piauí - PI)

In [28]:
query = """

WITH tb_regiao_metricas AS (

    SELECT
      t1.estadosigla AS estado
      , ROUND(AVG(t1.valordevenda),2) AS preco_medio
      , MIN(t1.valordevenda) AS preco_minimo
      , MAX(t1.valordevenda) AS preco_maximo

    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla

    ORDER BY
      AVG(t1.valordevenda) DESC
)
SELECT
  ROUND((1-(t3.preco_maximo/t2.preco_maximo))*100, 2) AS diff_percent
  , (t2.preco_maximo-t3.preco_maximo) AS diff_absoluta

FROM
  tb_regiao_metricas AS t2

JOIN
  tb_regiao_metricas AS t3

ON
  t2.estado = 'AC' AND t3.estado = 'PI'


"""
sqldf(query)

,diff_percent,diff_absoluta
0,21.49,1.64


Há uma diferença de 21% e R$ 1,64 entre os preços máximo que a gasolina já foi vendida no estado com maior preço médio e o estado com menor preço médio



---



### Query 9 - Análise apontando a diferença absoluta e percentual dos preços mínimo que a gasolina já foi vendida no estado com maior preço médio (Acre - AC) e o de menor preço médio (Piauí - PI)

In [27]:
query = """

WITH tb_regiao_metricas AS (

    SELECT
      t1.estadosigla AS estado
      , ROUND(AVG(t1.valordevenda),2) AS preco_medio
      , MIN(t1.valordevenda) AS preco_minimo
      , MAX(t1.valordevenda) AS preco_maximo

    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla

    ORDER BY
      AVG(t1.valordevenda) DESC
)

SELECT
  ROUND((1-(t3.preco_minimo/t2.preco_minimo))*100, 2) AS diff_percent
  , (t2.preco_minimo-t3.preco_minimo) AS diff_absoluta


FROM
  tb_regiao_metricas AS t2

JOIN
  tb_regiao_metricas AS t3

ON
  t2.estado = 'AC' AND t3.estado = 'PI'


"""
sqldf(query)

,diff_percent,diff_absoluta
0,23.57,1.53


Há uma diferença de 23% e R$ 1,53 entre os preços mínimo que a gasolina já foi vendida no estado com maior preço médio e o estado com menor preço médio



---



### Query 9 - Análise apontando a bandeira que cobra o maior preço médio por estado brasileiro

In [26]:
query = """

WITH tb_estado_bandeira_avg AS (
    SELECT
      t1.estadosigla AS estado
      , t1.bandeira AS bandeira
      , ROUND(AVG(t1.valordevenda), 2) AS preco_medio

    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla
      , t1.bandeira

    ORDER BY
      AVG(t1.valordevenda) DESC
)

    SELECT
      t2.estado
      , t2.bandeira
      , ROUND(MAX(t2.preco_medio), 2) AS maior_preco_medio

    FROM
      tb_estado_bandeira_avg AS t2

    GROUP BY
      t2.estado

    ORDER BY
      MAX(t2.preco_medio) DESC
"""
sqldf(query)

,estado,bandeira,maior_preco_medio
0,AC,ATEM' S,7.38
1,AM,BRANCA,6.94
2,RO,SABBÁ,6.56
3,PA,EQUADOR,6.42
4,PE,PETROBAHIA,6.38
5,RN,ALE,6.24
6,PR,ALE,6.19
7,RR,IPIRANGA,6.15
8,RS,SUL COMBUSTÍVEIS,6.14
9,AL,ALE,6.14




---



### Query 9 - Análise apontando a bandeira que cobra o menor preço médio por estado brasileiro

In [25]:
query = """

WITH tb_estado_bandeira_avg AS (
    SELECT
      t1.estadosigla AS estado
      , t1.bandeira AS bandeira
      , ROUND(AVG(t1.valordevenda), 2) AS preco_medio

    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla
      , t1.bandeira

    ORDER BY
      AVG(t1.valordevenda) DESC
)

    SELECT
      t2.estado
      , t2.bandeira
      , ROUND(MIN(t2.preco_medio), 2) AS menor_preco_medio

    FROM
      tb_estado_bandeira_avg AS t2

    GROUP BY
      t2.estado

    ORDER BY
      MIN(t2.preco_medio) DESC

"""
sqldf(query)

,estado,bandeira,menor_preco_medio
0,AC,EQUADOR,6.69
1,RO,RAIZEN,6.19
2,RR,BRANCA,6.09
3,AM,RZD DISTRIBUIDORA,5.99
4,CE,VIBRA,5.92
5,SE,BRANCA,5.89
6,RN,DISLUB,5.88
7,TO,BRANCA,5.85
8,BA,ALESAT,5.79
9,SC,STANG,5.75


### Query 10 - Análise apontando o município com maior preço médio de venda da gasolina do estado

In [24]:
query = """

WITH tb_estado_municipio_avg AS (
    SELECT
      t1.estadosigla AS estado
      , t1.municipio AS municipio
      , ROUND(AVG(t1.valordevenda), 2) AS preco_medio

    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla
      , t1.municipio

    ORDER BY
      AVG(t1.valordevenda) DESC
)

SELECT
  t2.estado
  , t2.municipio
  , ROUND(MAX(t2.preco_medio), 2) AS max_preco_medio

FROM
  tb_estado_municipio_avg AS t2

GROUP BY
  t2.estado

ORDER BY
  MAX(t2.preco_medio) DESC

"""
sqldf(query)

,estado,municipio,max_preco_medio
0,AM,TEFE,7.68
1,AC,CRUZEIRO DO SUL,7.41
2,RS,BAGE,6.78
3,MS,CORUMBA,6.77
4,PA,XINGUARA,6.73
5,AL,DELMIRO GOUVEIA,6.66
6,RO,PIMENTA BUENO,6.63
7,BA,PORTO SEGURO,6.61
8,ES,CACHOEIRO DE ITAPEMIRIM,6.54
9,RJ,TRES RIOS,6.40




---



### Query 11 - Análise apontando o município com menor preço médio de venda da gasolina do estado

In [31]:
query = """

WITH tb_estado_municipio_avg AS (
    SELECT
      t1.estadosigla AS estado
      , t1.municipio AS municipio
      , ROUND(AVG(t1.valordevenda), 2) AS preco_medio

    FROM
      df AS t1

    WHERE 1=1
      AND t1.produto = 'GASOLINA'

    GROUP BY
      t1.estadosigla
      , t1.municipio

    ORDER BY
      AVG(t1.valordevenda) DESC
)

SELECT
  t2.estado
  , t2.municipio
  , ROUND(MIN(t2.preco_medio),2) AS max_preco_medio

FROM
  tb_estado_municipio_avg AS t2

GROUP BY
  t2.estado

ORDER BY
  ROUND(MIN(t2.preco_medio),2) DESC

"""
sqldf(query)

,estado,municipio,max_preco_medio
0,AC,RIO BRANCO,6.76
1,RO,ARIQUEMES,6.26
2,RR,BOA VISTA,6.10
3,AM,MANACAPURU,6.07
4,RN,MOSSORO,5.95
5,SE,LAGARTO,5.86
6,CE,CAUCAIA,5.79
7,TO,ARAGUAINA,5.74
8,DF,BRASILIA,5.74
9,BA,SERRINHA,5.72


## Resultado:

- O preço médio geral de venda da gasolina entre os meses de Janeiro e Junho de 2024 foi de R$ 5,78 no Brasil.

- Quando fazemos uma análise geral entre o preço médio e os meses, vemos que com o passar dos meses os preços tendem a aumentar, tanto para os estados isoladamente, como os preços no geral.

- Existe uma variação notável nos preços médios da gasolina entre os estados. Entretanto, o que mais chama a atenção é a diferença entre os preços mínimo e máximo dentro de cada estado. Em muitos estados, essa diferença supera os 20% (em valores absolutos, superam a casa dos RS 1,40), indicando variação considerável nos preços de revenda(Vide QUERY 1.1);

- Quando paramos para olharmos, dentro do mesmo estado, há uma grande diferença entre o menor preço e o maior. A menor diferença está dentro do estado de Roraima, onde a diferença é de 8% (RS 0,55), ou seja, uma diferença baixa.

    - No entanto, a maior expressividade dessa diferença entá dentro do estado de São Paulo, onde a diferença percentual entre maior e menor preço é de  43,05% (RS 3,44) como apontado na análise da QUERY 1.1

- Referente aos preços médios, o ACRE (AC) possui o maior preço médio, de 6,99 Reais, sendo seguido pelo Amazonas (AM), com preço médio de 6,44 Reais. A diferença entre os dois é de 7,87%, ou seja,	diferença de RS 0,55

- A diferença mais expressiva é entre o estado com maior preço médio ACRE (AC) de RS 6,99 e o estado com menor preço médio, o Píaui com preço médio de R$ 5,57. A diferença entre os dois em porcentagem é de 20,32%, ou seja, valor absoluto de RS 1,42.

- Na análise podemos ver que o estado com maiores preços se é o Acre com um preço médio de RS 6,99. No entanto, quando paramos para olhar o municipio que possui o preço médio mais alto de todo o Brasil é o município de Tefé do estado do Amazonas. Tefé é considerado o meior preço da gasolina do país, com um preço médio de RS 7,68.

- Quando paramos para olhar o município do Acre com menor preço médio (Rio Branco com preço médio de RS 6,76), ele fica acima de quase todos os outros município de outros estados que possuem o maior preço médio em relação ao seu estado.







